In [1]:
import numpy as np
import pandas as pd
import re
from datetime import datetime

np.set_printoptions(threshold=100000)
pd.options.display.max_rows = 500
pd.options.display.max_columns = 100
pd.set_option('expand_frame_repr', False)

df = pd.read_excel('D:\工作\开竣工数据整理\删去2017年12月30后数据-3.xls')
df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3108 entries, 0 to 3107
Data columns (total 24 columns):
id        3108 non-null int64
用地单位      3094 non-null object
宗地号       3108 non-null object
合同号       3108 non-null object
合同字号      3108 non-null object
合同签订日     3108 non-null object
合同总地价     3108 non-null float64
出让金       3108 non-null float64
开发费       3108 non-null float64
配套费       3108 non-null float64
用地面积      3108 non-null float64
供应方式      3081 non-null object
协议出让原     0 non-null float64
土地类别      2956 non-null object
土地使用年     3108 non-null object
项目名称      670 non-null object
总建筑面积     3107 non-null float64
开工日期      689 non-null object
竣工日期      2072 non-null object
局别        3108 non-null object
宗地号_1     0 non-null float64
竣工日期_增    29 non-null object
是否建成      128 non-null object
开工日期_增    7 non-null object
dtypes: float64(8), int64(1), object(15)
memory usage: 607.0+ KB


In [2]:
# df_nodup = df.drop_duplicates(keep='first', subset=['宗地号'])

def date_combine(date1, date2):
    return date2 if len(str(date1))<5 else date1

zongdi_list = list(set(list(df['宗地号'])))

df['开工日期'] = df.apply(lambda x: date_combine(x['开工日期'], x['开工日期_增']), axis=1)
df['竣工日期'] = df.apply(lambda x: date_combine(x['竣工日期'], x['竣工日期_增']), axis=1)
df['开工日期'] = df['开工日期'].astype('str')
df['竣工日期'] = df['竣工日期'].astype('str')


df['开工日期'] = df['开工日期'].map(lambda x: '0000' if x=='nan' else x)
df['竣工日期'] = df['竣工日期'].map(lambda x: '0000' if x=='nan' else x)
df['合同字号_1'] = df['合同字号']
df['合同字号_1'] = df['合同字号_1'].map(lambda x: x.replace('合', 'a'))
df = df.sort_values(by=['宗地号', '合同签订日', '合同字号_1'])

j1 = []
j2 = []
k1 = []
k2 = []
jcc =[]

for i in zongdi_list:
    df_i = df[df.宗地号==i]
    
    jungong_list = list(df_i['竣工日期'])   # 获取每个宗地下的所有竣工日期
    jungong_init = jungong_list[0]
    jungong_bu = sorted(jungong_list)[-1]
    i1 = jungong_init
    i2 = jungong_bu
    j1.append(i1)
    j2.append(i2)
    
    
    kungong_list = list(df_i['开工日期'])
    kungong_init = kungong_list[0]
    kungong_bu = sorted(kungong_list)[-1]
    ik1 = kungong_init
    ik2 = kungong_bu
    k1.append(ik1)
    k2.append(ik2)
    
    jc_list = list(df_i['是否建成'])

    jc = '已建成' if '已建成' in jc_list else ''
    jcc.append(jc)
    

    
dfx_dict = {'宗地号': zongdi_list, '开工日期': k1, '开工日期_补': k2,  '竣工日期': j1, '竣工日期_补': j2, '是否建成': jcc}
dfx = pd.DataFrame(dfx_dict)


def comp(x, y):
    z = y
    if x == y:
        z = ''
    return z

dfx['竣工日期_补'] = dfx.apply(lambda x: comp(dfx['竣工日期'], dfx['竣工日期_补']), axis =1)
dfx.to_excel('D:\工作\开竣工数据整理\最终确定3429_转换.xls')

In [3]:
def comp(x, y): 
    return '' if x==y else y


dfx['竣工日期_补'] = dfx.apply(lambda x: comp(x['竣工日期'], x['竣工日期_补']), axis=1) 
dfx['开工日期_补'] = dfx.apply(lambda x: comp(x['开工日期'], x['开工日期_补']), axis=1) 
dfx = dfx.applymap(lambda x: '' if x=='0000' else x)

dfx.loc[:, '开工日期': '竣工日期_补'].applymap(lambda x: datetime.strptime(x.replace('00:00:00', '').replace(', ', ''), '%Y-%m-%d') if len(x)>4 else '')
dfx = dfx.sort_values(by=['宗地号'])
dfx.to_excel('D:\工作\开竣工数据整理\删去2017年12月30后数据_转换5.xlsx')

In [9]:
### 合并所有合同
import numpy as np
import pandas as pd

df = pd.read_excel('D:\工作\开竣工数据整理\初始11777.xlsx')
df2 = pd.read_excel('D:\工作\开竣工数据整理\开竣工日期补-f.xlsx')

df_main = df[(df['合同字号']=='合')|(df['合同字号']=='划拨')|(df['合同字号']=='协')|(df['合同字号']=='资')]
df_main = df_main.sort_values(by=['合同签订日期'])
df_main = df_main.drop_duplicates(subset=['宗地号'], keep='first')
df_main



,用地单位,宗地号,合同号,合同字号,合同签订日期,合同总地价款,出让金,开发费,配套费,用地面积,供应方式,协议出让原因,土地类别,土地使用年限,项目名称,总建筑面积,开工日期,竣工日期,局别
898,深圳市劲拓自动化设备有限公司,A122-0259,2007-4815,协,2007-07-04,7.561370e+05,1.134210e+05,0.0,6.427160e+05,14290.40,协议,NaN,一类工业用地,50年0月,NaN,0.00,NaN,NaN,宝安管理局
4253,"博时基金管理有限公司,南方基金管理有限公司",B203-0020,2007-0041,合,2007-07-04,2.879122e+08,4.318683e+07,728712.0,2.439967e+08,7260.06,协议,NaN,商业性办公用地,50年0月,NaN,0.00,NaN,2009-06-28,福田管理局
10032,深圳市众联业贸易有限公司,T102-0037,2007-0099,合,2007-07-04,2.846921e+06,3.986900e+05,188985.0,2.259246e+06,1889.85,协议,NaN,商业性办公用地,50年0月,NaN,0.00,NaN,2009-07-02,福田管理局
1012,深圳市机场（集团）有限公司,A201-0129,2007-4034,合,2007-07-10,2.557476e+07,3.836213e+06,0.0,2.173854e+07,53280.74,协议,NaN,机场用地(A号航站楼北侧远机位停机坪）,50年0月,NaN,0.00,NaN,2010-01-10,宝安管理局
1017,深圳市机场（集团）有限公司,A201-0134,2007-4028,合,2007-07-10,9.567576e+06,1.435136e+06,0.0,8.132440e+06,19932.45,协议,NaN,机场用地(国际候机楼）,50年0月,NaN,0.00,NaN,2010-01-10,宝安管理局
659,深圳市水务（集团）有限公司,A112-0075,2007-4023,合,2007-07-12,3.244224e+06,4.866340e+05,413595.0,2.343995e+06,6893.25,协议,NaN,供应设施用地,50年0月,NaN,0.00,NaN,2009-01-12,宝安管理局
10742,深圳市南山区人民政府,T203-0073,2007-0102,合,2007-07-16,0.000000e+00,0.000000e+00,0.0,0.000000e+00,7565.69,协议,NaN,市属办公用地,50年0月,NaN,0.00,NaN,2009-12-31,福田管理局
11566,深圳市人民政府,T408-0019,2007-0095,合,2007-07-16,0.000000e+00,0.000000e+00,0.0,0.000000e+00,15156.04,协议,NaN,交通设施用地,50年0月,NaN,0.00,NaN,2009-06-24,福田管理局
684,深圳市宝安区教育局,A113-0006,2007-4148,合,2007-07-24,0.000000e+00,0.000000e+00,0.0,0.000000e+00,71198.06,协议,NaN,中学用地,50年0月,NaN,0.00,NaN,2010-07-24,宝安管理局
146,深圳市宝安区建筑工务局,A004-0104,2007-4164,合,2007-07-25,0.000000e+00,0.000000e+00,0.0,0.000000e+00,94536.53,协议,NaN,口岸设施用地,50年0月,NaN,0.00,NaN,NaN,宝安管理局


In [10]:
dfx = df_main.merge(df2, on='宗地号', how='left' )
def date_clean(x, y):
    z = x
    if x != x:
        z= y
    return z
dfx['开工日期'] = dfx.apply(lambda x: date_clean(x.开工日期_x, x.开工日期_y), axis=1)
dfx['竣工日期'] = dfx.apply(lambda x: date_clean(x.竣工日期_x, x.竣工日期_y), axis=1)



In [12]:
dfx.drop(['开工日期_x','开工日期_y', '竣工日期_x','竣工日期_y','序号' ], inplace=True, axis=1)


KeyError: "labels ['开工日期_x' '开工日期_y' '竣工日期_x' '竣工日期_y' '序号'] not contained in axis"

In [13]:
dfx.to_excel('D:\工作\开竣工数据整理\开竣工整理-f.xlsx', index=False)